In [0]:
from google.colab import drive
drive.mount('gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at gdrive


In [0]:
%cd '/content/gdrive/My Drive/kisti_final_project/Waveform/jjw'

/content/gdrive/My Drive/kisti_final_project/Waveform/jjw


In [0]:
!pip install soundfile

In [0]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import rmsprop

Using TensorFlow backend.


In [0]:
def extract_feature(file_name,mfcc=True,chroma=True,mel=True):
    sf = soundfile.SoundFile(file_name)
    X = sf.read(dtype = 'float32')
    if X.ndim > 1:
        return np.array([])
    sr = sf.samplerate
    if chroma:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sr,n_mfcc=40).T,axis=0)
        result = np.hstack((result,mfccs))
    if chroma:
        chromas = np.mean(librosa.feature.chroma_stft(S=stft,sr=sr).T,axis=0)
        result = np.hstack((result,chromas))
    if mel:
        mels = np.mean(librosa.feature.melspectrogram(X,sr=sr).T,axis=0)
        result = np.hstack((result,mels))
    sf.close()
    return result

In [0]:
path = '/content/gdrive/My Drive/kisti_final_project/dataset/dataset_70db-all-in-one/*.wav'
emotions = {'0':'부정적','1':'긍정적'}
x_data,y_data = [],[]
for f in glob.glob(path):
    fn = os.path.basename(f)
    emo = fn[2]
    feature = extract_feature(f)
    x_data.append(feature)
    y_data.append(emo)
x_train,x_test,y_train,y_test= train_test_split(np.array(x_data), y_data, test_size=0.2, random_state=1229)

In [0]:
print('X_train : ',x_train.shape,'X_test : ',x_test.shape,'\n Y_train : ',len(y_train),'Y_test : ',len(y_test))

X_train :  (479, 180) X_test :  (120, 180) 
 Y_train :  479 Y_test :  120


In [0]:
y_train_cat = to_categorical(np.array(y_train))
y_test_cat = to_categorical(np.array(y_test))

In [0]:
print('X_train : ',x_train.shape,' X_test : ',x_test.shape,'\n Y_train : ',len(y_train_cat),'  Y_test : ',len(y_test_cat))

X_train :  (479, 180)  X_test :  (120, 180) 
 Y_train :  479   Y_test :  120


In [0]:
def new_CNN():
    ### CNN model, referred to the model C in the report
    model = Sequential()
    model.add(Conv1D(8, 5,padding='same', input_shape=(180, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(16, 5,padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(32, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(16, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(2))
    model.add(Activation('softmax'))
    opt = keras.optimizers.Adam(lr=0.0001, decay=1e-6)
    model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
    return model

In [0]:
def new_CNN_al():
    ### CNN model, referred to the model C in the report
    model = Sequential()
    model.add(Conv1D(8, 5,padding='same', input_shape=(180, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(16, 5,padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(32, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(16, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(2))
    model.add(Activation('softmax'))
    opt = keras.optimizers.Adam(lr=0.0001, decay=1e-6)
    model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
    return model

In [0]:
model_WC = new_CNN()
wc_fit = model_WC.fit(np.expand_dims(x_train,-1),y_train_cat,
                     validation_data=(np.expand_dims(x_test,-1), y_test_cat), 
                     epochs=350, shuffle=True)

In [0]:
model_WCA = new_CNN_al()
wc_fit = model_WCA.fit(np.expand_dims(x_train,-1),y_train_cat,
                     validation_data=(np.expand_dims(x_test,-1), y_test_cat), 
                     epochs=350, shuffle=True)




Train on 479 samples, validate on 120 samples
Epoch 1/350





479/479 [==============================] - 1s 2ms/step - loss: 2.5780 - acc: 0.6722 - val_loss: 1.3891 - val_acc: 0.6417
Epoch 2/350
479/479 [==============================] - 0s 372us/step - loss: 1.6560 - acc: 0.5783 - val_loss: 0.6874 - val_acc: 0.5833
Epoch 3/350
479/479 [==============================] - 0s 360us/step - loss: 1.5579 - acc: 0.5428 - val_loss: 0.8062 - val_acc: 0.6417
Epoch 4/350
479/479 [==============================] - 0s 372us/step - loss: 1.3588 - acc: 0.5887 - val_loss: 0.7249 - val_acc: 0.6417
Epoch 5/350
479/479 [==============================] - 0s 364us/step - loss: 1.4752 - acc: 0.5699 - val_loss: 0.7056 - val_acc: 0.6333
Epoch 6/350
479/479 [==============================] - 0s 361us/step - loss: 1.3359 - acc: 0.5950 - val_loss: 0.7273 - val_acc: 0.6417
Epoch 7/350
479/479 [==============================] - 0s 346us/step - loss: 1.2149 - acc: 0.6096 - val_loss: 0.6774 - val_acc: 0.6417
Epo

In [0]:
model_WCA.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 180, 8)            48        
_________________________________________________________________
activation_6 (Activation)    (None, 180, 8)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 180, 16)           656       
_________________________________________________________________
activation_7 (Activation)    (None, 180, 16)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 180, 16)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 22, 16)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 22, 32)           

In [0]:
model_WC.summary()

In [0]:
%ls '/content/gdrive/My Drive/kisti_final_project/dataset/test/'

0여자부정_후보01.wav       0여자부정후보1.wav  0여자부정후보8.wav
0여자부정_후보02.wav       0여자부정후보2.wav  1여자긍정_후보02_애매.wav
0여자부정_후보03.wav       0여자부정후보5.wav  1남자긍정_후보04_애매.wav
0여자부정_후보04_애매.wav  0여자부정후보7.wav  1여자긍정_후보05_애매.wav


In [0]:
t1,emo_check = [],[]
for x in glob.glob('/content/gdrive/My Drive/kisti_final_project/dataset/test/*.wav'):
  fn = os.path.basename(x)
  emo = fn[0]
  feature = extract_feature(x)
  t1.append(feature)
  emo_check.append(emo)

In [0]:
t2 = np.array(t1)
emo_check = to_categorical(np.array(emo_check))
print(t2.shape)
print(type(t2))

(12, 180)
<class 'numpy.ndarray'>


In [0]:
s = model_WCA.evaluate(np.expand_dims(t2,-1),emo_check,batch_size=100)

12/12 [==============================] - 0s 510us/step


In [0]:
print(s[1])

0.6666666865348816


In [0]:
def CNN_Predict(n):
    pre_C = model_WC.predict(np.expand_dims(n,-1),batch_size=100)
    CNN_list = []
    for i,x in enumerate(pre_C):
        CNN_list.append('CNN 모델로 판단할 경우 {0}번 음성은 {1:.2f}확률로 비호감이며,{2:.2f}확률로 호감입니다.'.format(i+1,x[0]*100,x[1]*100))
    return CNN_list

In [0]:
model_WCA.save('../CNN_by_Wav.h5')